# Projeto Final — Análise Silver
## Análise de Reclamações Financeiras

### Objetivo da Camada Silver
Nesta camada realizamos análises exploratórias e descritivas para responder às perguntas de negócio:
- Perfil dos consumidores que mais reclamam
- Tipos de problemas mais recorrentes
- Desempenho das instituições financeiras
- Panorama geral de satisfação

Os insights gerados aqui servirão de base para a camada Gold (análises aprofundadas e comparativos estratégicos).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

agibank_palette = ["#0064F5", "#77DF40", "#FFD600", "#0033B0", "#000F4A", "#C9FF90"]
sns.set_palette(agibank_palette)
plt.style.use("seaborn-v0_8-whitegrid")
sns.set_context("talk")

plt.rcParams["axes.labelcolor"] = "#000F4A"
plt.rcParams["xtick.color"] = "#000F4A"
plt.rcParams["ytick.color"] = "#000F4A"
plt.rcParams["text.color"] = "#000F4A"

In [ ]:
arquivo = "../../data/silver/consumidor_gov_bronze_v2.csv"

df = pd.read_csv(arquivo, sep=";", encoding="utf-8", low_memory=False)

print("Dimensão da base Silver:", df.shape)
print(f"Carregados {df.shape[0]:,} registros e {df.shape[1]} colunas".replace(",", "."))

In [ ]:
print("Primeiras 5 linhas da base:\n")
display(df.head())

In [ ]:
print("Informações gerais das colunas:\n")
df.info()

In [ ]:
print("Estatísticas descritivas:\n")
display(df.describe())

In [ ]:
nulos = df.isna().mean().sort_values(ascending=False) * 100

print("Top 10 colunas com mais valores nulos:\n")
display(nulos.head(10))

In [ ]:
print("=" * 80)
print("ANALISE DE COLUNAS TEMPORAIS")
print("=" * 80)

palavras_chave = ['ano', 'mês', 'mes', 'data', 'prazo', 'tempo', 'processed_at', 'file_month']

colunas_temporais = []
for col in df.columns:
    col_lower = col.lower()
    if any(palavra in col_lower for palavra in palavras_chave):
        if 'source' not in col_lower:
            colunas_temporais.append(col)

print(f"\nColunas temporais encontradas: {len(colunas_temporais)}")
print("-" * 80)
for col in colunas_temporais:
    print(f"   {col} ({df[col].dtype})")

problemas = []

for col in colunas_temporais:
    tipo_atual = str(df[col].dtype)
    
    if col in ['Data Abertura', 'Data Resposta', 'Data Finalização', 'Prazo Resposta', 'processed_at']:
        if tipo_atual != 'datetime64[ns]':
            problemas.append({
                'Coluna': col, 
                'Tipo Atual': tipo_atual, 
                'Tipo Esperado': 'datetime64[ns]'
            })

print("\n\nPROBLEMAS IDENTIFICADOS")
print("-" * 80)

if problemas:
    print(f"\n{len(problemas)} colunas precisam ser corrigidas:\n")
    
    df_problemas = pd.DataFrame(problemas)
    df_problemas.index = range(1, len(df_problemas) + 1)
    
    display(df_problemas)
    
    print("\nColunas que estao CORRETAS e NAO precisam ser alteradas:")
    print("   Ano Abertura (int64)")
    print("   Mes Abertura (int64)")
    print("   Tempo Resposta (float64)")
    print("   file_month (object)")
    
else:
    print("\nTodas as colunas temporais estao corretas!")

print("\n" + "=" * 80)

In [ ]:
print("=" * 80)
print("ANALISE DE TIPOS DE DADOS - TODAS AS COLUNAS")
print("=" * 80)

print(f"\nTotal de colunas: {len(df.columns)}")
print(f"Total de registros: {len(df):,}")

analise_colunas = []

for col in df.columns:
    tipo = str(df[col].dtype)
    nao_nulos = df[col].notna().sum()
    nulos = df[col].isna().sum()
    perc_nulos = round((nulos / len(df)) * 100, 2)
    valores_unicos = df[col].nunique()
    amostra = str(df[col].dropna().iloc[0]) if df[col].notna().any() else 'N/A'
    
    analise_colunas.append({
        'Coluna': col,
        'Tipo': tipo,
        'Nulos': nulos,
        '% Nulos': perc_nulos,
        'Valores Unicos': valores_unicos,
        'Exemplo': amostra[:50]
    })

df_analise = pd.DataFrame(analise_colunas)

print("\n")
display(df_analise)

print("\n" + "=" * 80)
print("RESUMO POR TIPO DE DADO")
print("=" * 80)

tipos_resumo = df.dtypes.value_counts()
print("\n")
for tipo, qtd in tipos_resumo.items():
    print(f"{tipo}: {qtd} colunas")

print("\n" + "=" * 80)

In [ ]:
print("=" * 80)
print("RELATORIO DE QUALIDADE DE DADOS - TIPOS E VALORES")
print("=" * 80)

print("\n" + "=" * 80)
print("PROBLEMAS CRITICOS - DEVEM SER CORRIGIDOS")
print("=" * 80)

criticos = []

for col in df.columns:
    tipo_atual = str(df[col].dtype)
    valores_unicos = df[col].nunique()
    
    if tipo_atual == 'object':
        valores_com_espaco = df[col].dropna().astype(str).str.strip() != df[col].dropna().astype(str)
        if valores_com_espaco.any():
            exemplos = df[col][df[col].notna()].unique()[:5]
            criticos.append({
                'Coluna': col,
                'Tipo Atual': tipo_atual,
                'Problema': f'Valores com espaços extras',
                'Acao Necessaria': 'Aplicar .str.strip() para remover espaços'
            })
    
    if tipo_atual == 'object' and valores_unicos > 10:
        amostra = str(df[col].dropna().iloc[0])
        if '/' in amostra or '-' in amostra:
            tem_numeros = any(char.isdigit() for char in amostra)
            if tem_numeros and len(amostra) >= 8:
                criticos.append({
                    'Coluna': col,
                    'Tipo Atual': tipo_atual,
                    'Problema': f'Parece ser data mas esta como texto. Exemplo: {amostra}',
                    'Acao Necessaria': 'Converter para datetime64[ns]'
                })
    
    if tipo_atual == 'float64':
        valores_nao_nulos = df[col].dropna()
        if len(valores_nao_nulos) > 0:
            sao_inteiros = (valores_nao_nulos % 1 == 0).all()
            if sao_inteiros and valores_unicos <= 20:
                valores = sorted(df[col].dropna().unique())
                criticos.append({
                    'Coluna': col,
                    'Tipo Atual': tipo_atual,
                    'Problema': f'Representa valores inteiros {valores} mas esta como float',
                    'Acao Necessaria': 'Converter para int64 (tratar nulos)'
                })
    
    if valores_unicos == 1:
        valor = df[col].unique()[0]
        criticos.append({
            'Coluna': col,
            'Tipo Atual': tipo_atual,
            'Problema': f'Todos registros tem valor fixo: {valor}',
            'Acao Necessaria': 'Verificar se coluna esta sendo calculada/preenchida corretamente'
        })

if criticos:
    df_criticos = pd.DataFrame(criticos)
    df_criticos.index = range(1, len(df_criticos) + 1)
    print()
    display(df_criticos)
else:
    print("\nNenhum problema critico encontrado!")

print("\n" + "=" * 80)
print("MELHORIAS OPCIONAIS - OTIMIZACAO")
print("=" * 80)

opcionais = []

for col in df.columns:
    tipo_atual = str(df[col].dtype)
    valores_unicos = df[col].nunique()
    
    if tipo_atual == 'object' and valores_unicos == 2:
        valores = set(df[col].dropna().unique())
        if valores == {'S', 'N'}:
            opcionais.append({
                'Coluna': col,
                'Tipo Atual': tipo_atual,
                'Sugestao': 'Converter S/N para boolean (True/False)',
                'Beneficio': 'Melhor performance e clareza'
            })
    
    if tipo_atual == 'object' and 2 <= valores_unicos <= 50:
        tamanho_atual = df[col].memory_usage(deep=True) / 1024 / 1024
        if tamanho_atual > 1:
            opcionais.append({
                'Coluna': col,
                'Tipo Atual': tipo_atual,
                'Sugestao': f'Converter para category ({valores_unicos} valores unicos)',
                'Beneficio': f'Economia de memoria (~{tamanho_atual:.1f}MB)'
            })

if opcionais:
    df_opcionais = pd.DataFrame(opcionais)
    df_opcionais.index = range(1, len(df_opcionais) + 1)
    print()
    display(df_opcionais)
else:
    print("\nNenhuma melhoria opcional identificada")

print("\n" + "=" * 80)
print("RESUMO EXECUTIVO")
print("=" * 80)
print(f"\nTotal de problemas criticos: {len(criticos)}")
print(f"Total de melhorias opcionais: {len(opcionais)}")

print("\n" + "=" * 80)